# DAY 8 : Unity Catalog Governance

In [0]:
%sql
-- STEP 1: Set the context
USE CATALOG workspace;
CREATE SCHEMA IF NOT EXISTS governance_lab;
USE SCHEMA governance_lab;

-- STEP 2: Create a MANAGED Delta Table
-- Databricks manages the underlying files
CREATE TABLE IF NOT EXISTS managed_sensor_data (
    sensor_id INT,
    temperature DOUBLE,
    timestamp TIMESTAMP
) USING DELTA;

In [0]:
%sql
-- Grant read-only access to a specific group/user
GRANT USAGE ON SCHEMA governance_lab TO `users`;
GRANT SELECT ON TABLE managed_sensor_data TO `users`;

-- Revoke permissions if a user changes roles
REVOKE ALL PRIVILEGES ON TABLE managed_sensor_data FROM `intern_account`;

-- Check who has access
SHOW GRANTS ON TABLE managed_sensor_data;

In [0]:
%sql
-- Create a View that filters sensitive data
-- Only shows temperatures above 100 for 'Critical' alerts
CREATE OR REPLACE VIEW critical_alerts AS
SELECT sensor_id, temperature
FROM managed_sensor_data
WHERE temperature > 100;

-- Lineage: Unity Catalog automatically tracks that 'critical_alerts' 
-- depends on 'managed_sensor_data'.

In [0]:
import pandas as pd

# Define the 3-level path
table_path = "workspace.governance_lab.critical_alerts"

try:
    # Attempt to read the view into a DataFrame
    df = spark.sql(f"SELECT * FROM {table_path}").toPandas()
    print("Governance Lab Success: View loaded into Pandas.")
    print(df.head())
except Exception as e:
    print(f"Access Denied or Table Missing: {e}")

Governance Lab Success: View loaded into Pandas.
Empty DataFrame
Columns: [sensor_id, temperature]
Index: []
